In [7]:
import os

from dotenv import load_dotenv
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma, Milvus
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()

True

In [8]:
# Define the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the existing vector store with the embedding function
db = Milvus(embedding_function=embeddings, collection_name="novels",
            connection_args={"host": "127.0.0.1", "port": "19530"})

# Create a retriever for querying the vector store
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Create a ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o")

In [9]:
# Contextualize question prompt
contextualize_q_system_prompt = "Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages([("system", contextualize_q_system_prompt), 
                                                           MessagesPlaceholder("chat_history"), 
                                                           ("human", "{input}"),])

# Create a history-aware retriever
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

# Answer question prompt
qa_system_prompt = "You are a helpful book recommender. Use the following pieces of retrieved context for books to recommend titles based on the request. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages([("system", qa_system_prompt), 
                                              MessagesPlaceholder("chat_history"), 
                                              ("human", "{input}")])

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` - feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [10]:
# Function to simulate a continual chat
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)

    while True:
        query = input("You: ")

        if query.lower() == "exit":
            break

        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        
        # Display the AI's response
        print(f"AI: {result['answer']}")
        
        # Update the chat history
        chat_history.append(HumanMessage(content=query))
        chat_history.append(SystemMessage(content=result["answer"]))

# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()

Start chatting with the AI! Type 'exit' to end the conversation.
AI: I recommend "The Adventures of Huckleberry Finn" by Mark Twain. Set in the Mississippi Valley, it captures the spirit of America forty to fifty years before its publication, exploring themes of freedom, friendship, and social issues. It's a classic American novel that offers both adventure and insight into the country's history.


KeyboardInterrupt: Interrupted by user